In [27]:
header = """
\\begin{figure}[H]
  {
    \\setlength{\\tabcolsep}{3.0pt}
    \\setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
    \\begin{adjustbox}{width=15cm,center}
      \\begin{tikzpicture}
"""

footer = """
      \\end{tikzpicture}
    \\end{adjustbox}
  }\\caption{%PLANET_NAME%}
\\end{figure}
"""

In [83]:
"""
backgroundColor1ForPlanets .BYTE $09,$0B,$07,$0E,$0D
stored in currentPlanetBackgroundClr1 as an index 
and
backgroundColor2ForPlanets .BYTE $0E,$10,$01,$07,$10
stored in currentPlanetBackgroundClr2 as an index into:

backgroundColorsForPlanets         .BYTE BLACK,WHITE,RED,CYAN,PURPLE,GREEN,BLUE,YELLOW      
                                   .BYTE ORANGE,BROWN,LTRED,GRAY1,GRAY2,LTGREEN,LTBLUE,GRAY3
"""
"""
surfaceColorsForPlanets    .BYTE LTGREEN,BROWN,LTRED,GRAY2,LTRED,WHITE,WHITE
"""
planet_colors = {
    'planet1Charset' : ['brown','lightblue','lightgreen'],
    'planet2Charset' : ['gray','lightred','brown'],
    'planet3Charset' : ['yellow','white', 'lightred'],
    'planet4Charset' : ['lightblue','yellow', 'darkgray'],
    'planet5Charset' : ['lightgreen','lightred','lightred'],
}



In [84]:
import re
colors = {
    "00": "",
    "01": "\\BACKGROUNDONE",
    "10": "\\BACKGROUNDTWO",
    "11": "\\CHARCOLOR",
}
charsets_files = ["../iridisalpha/src/graphics/planet_textures.asm"]
charsets = {}
for charsets_file in charsets_files:
    input_file = open(charsets_file,'r')
    charset_data = []
    curr_charset = []
    for l in input_file.readlines():
        if "CHARACTER" in l or "planet" in l:
            if charset_data:
                curr_charset += [charset_data]
            charset_data = []
        if "planet" in l:
            if curr_charset:
                charsets[charset_name] = curr_charset
            charset_name = l.strip()
            curr_charset = []
            continue

        m = re.findall(r"[0-1]{8}",l)
        if not m:
            continue
        bits = m[0]
        for i in range(0,7,2):
            bitpair = bits[i:i+2]
            charset_data += [colors[bitpair]]
            charset_data += [colors[bitpair]]
    charsets[charset_name] = curr_charset


In [122]:
def fillCharacter(c, i=0, BIT_WIDTH=8):
    output = ""
    gh = BIT_WIDTH - 1
    for j,v in enumerate(c):
        if not v:
            continue
        x = (i * BIT_WIDTH) + ((j % BIT_WIDTH) if j >= (BIT_WIDTH - 1) else j)
        y = (gh - int(j/BIT_WIDTH)) if j >= gh else gh
        output += f'\t\\fill[{v}] ({x},{y}) rectangle ++ (1,1);\n'
    return output

def createPlanetSurface(charset_name, byte_ref, surface):
    PLANET_WIDTH = 32
    BIT_WIDTH = 8
    gl = PLANET_WIDTH * BIT_WIDTH

    output = header + '\n'
    output += f"\\def\BACKGROUNDONE{{{planet_colors[charset_name][0]}}}\n"
    output += f"\\def\BACKGROUNDTWO{{{planet_colors[charset_name][1]}}}\n"
    output += f"\\def\CHARCOLOR{{{planet_colors[charset_name][2]}}}\n"

    grid = f"\\draw[step=1.0,gray,thin] (0,0) grid ({gl},{BIT_WIDTH});"
    output += f"\t{grid}\n"
    charset = charsets[charset_name]
    for i in range(0, PLANET_WIDTH):
        c = charset[byte_ref[1] if i % 2 else byte_ref[0]]
        output += fillCharacter(c, i, BIT_WIDTH)
    output += footer.replace("%PLANET_NAME%", charset_name.replace("_","\_")
                             + " " + surface)
    return output

SEA = [0,2]
LAND = [1,3]
surfaces = [("Sea",SEA), ("Land",LAND)]
for nm in planet_colors:
    for surface, byte_ref in surfaces:
        output = createPlanetSurface(nm, byte_ref, surface)
        output_file = open(f"../src/planets/{nm}_{surface.lower()}.tex",'w')
        output_file.write(output)
        output_file.close()



In [123]:
def createTile(planet_name, index, ref):
    header = """
    \\begin{figure}[H]
      {
        \\setlength{\\tabcolsep}{3.0pt}
        \\setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
        \\begin{adjustbox}{width=2cm,center}
          \\begin{tikzpicture}
    """
    BIT_WIDTH = 8
    output = header + '\n'
    output += f"\\def\BACKGROUNDONE{{{planet_colors[planet_name][0]}}}\n"
    output += f"\\def\BACKGROUNDTWO{{{planet_colors[planet_name][1]}}}\n"
    output += f"\\def\CHARCOLOR{{{planet_colors[planet_name][2]}}}\n"
    
    grid = f"\\draw[step=1.0,gray,thin] (0,0) grid ({BIT_WIDTH},{BIT_WIDTH});"
    output += f"\t{grid}\n"

    charset = charsets[planet_name]
    c = charset[index]
    output += fillCharacter(c)

    output += footer.replace("%PLANET_NAME%", planet_name.replace("_","\_")
                             + " \\" + ref)
    return output

SEA = [0,2]
LAND = [1,3]
surfaces = [("Sea",SEA), ("Land",LAND)]
for planet_name in planet_colors:
    for surface, byte_ref in surfaces:
        for i in byte_ref:
            ref = f"${hex(0x40 + i)[2:].upper()}"
            output = createTile(planet_name, i, ref)
            of_name = f"../src/planets/{planet_name}_{ref}.tex"
            print(of_name)
            with open(of_name,'w') as output_file:
                output_file.write(output)


../src/planets/planet1Charset_$40.tex
../src/planets/planet1Charset_$42.tex
../src/planets/planet1Charset_$41.tex
../src/planets/planet1Charset_$43.tex
../src/planets/planet2Charset_$40.tex
../src/planets/planet2Charset_$42.tex
../src/planets/planet2Charset_$41.tex
../src/planets/planet2Charset_$43.tex
../src/planets/planet3Charset_$40.tex
../src/planets/planet3Charset_$42.tex
../src/planets/planet3Charset_$41.tex
../src/planets/planet3Charset_$43.tex
../src/planets/planet4Charset_$40.tex
../src/planets/planet4Charset_$42.tex
../src/planets/planet4Charset_$41.tex
../src/planets/planet4Charset_$43.tex
../src/planets/planet5Charset_$40.tex
../src/planets/planet5Charset_$42.tex
../src/planets/planet5Charset_$41.tex
../src/planets/planet5Charset_$43.tex


In [96]:
x = 12
o = 0x40
r = f"${hex(o + x)[2:].upper()}"
r

'$4C'